# Feed Forward Neural Network for Credit Card Applications (Model)

## Data Preprocessing for Neural Network

In [2]:
import pandas as pd

#Load the dataset to be used
df1 = pd.read_csv("updatedMergedApplicationCreditRecords.csv")
df1.head()

,Unnamed: 0,ID,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,YEARS_OLD,DAYS_EMPLOYED,MONTHS_BALANCE,STATUS
0,0,5008804,Y,Y,427500.0,Working,Higher education,Civil marriage,Rented apartment,32.0,4542.0,-0.0,1
1,1,5008804,Y,Y,427500.0,Working,Higher education,Civil marriage,Rented apartment,32.0,4542.0,1.0,1
2,2,5008804,Y,Y,427500.0,Working,Higher education,Civil marriage,Rented apartment,32.0,4542.0,2.0,1
3,3,5008804,Y,Y,427500.0,Working,Higher education,Civil marriage,Rented apartment,32.0,4542.0,3.0,1
4,4,5008804,Y,Y,427500.0,Working,Higher education,Civil marriage,Rented apartment,32.0,4542.0,4.0,1


In [3]:
#Begin by removing duplicates and pruning columns we determined in the Unsupervised Learning step to be unhelpful
df1 = df1.drop_duplicates("ID")
df1 = df1.drop(columns=["Unnamed: 0", "ID", "FLAG_OWN_CAR", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "NAME_HOUSING_TYPE", "YEARS_OLD"])
df1.head()

,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,DAYS_EMPLOYED,MONTHS_BALANCE,STATUS
0,Y,427500.0,4542.0,-0.0,1
16,Y,427500.0,4542.0,-0.0,1
31,Y,112500.0,1134.0,-0.0,1
61,Y,270000.0,3051.0,-0.0,1
66,Y,270000.0,3051.0,22.0,1


In [4]:
#Make a OneHotEncoder for our FLAG_OWN_REALTY column

from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix

#Start OneHotEncoding Process
encoder = make_column_transformer((OneHotEncoder(), ["FLAG_OWN_REALTY"]), remainder="passthrough")
encoded = encoder.fit_transform(df1)
encoded_dataset = pd.DataFrame(encoded, columns=encoder.get_feature_names_out())

#Fix up the encoded column names
encoded_dataset.rename(columns= {"onehotencoder__FLAG_OWN_REALTY_N":"NO_REALTY",
                                "onehotencoder__FLAG_OWN_REALTY_Y":"YES_REALTY",
                                "remainder__AMT_INCOME_TOTAL":"TOTAL_INCOME",
                                "remainder__DAYS_EMPLOYED":"DAYS_EMPLOYED",
                                "remainder__MONTHS_BALANCE":"MONTHS_BALANCE",
                                "remainder__STATUS":"STATUS"}, inplace = True)

#encoded_dataset.head()

#Begin normalization process
minMax = MinMaxScaler()
#encoded_dataset[["TOTAL_INCOME", "DAYS_EMPLOYED", "MONTHS_BALANCE"]] = minMax.fit_transform(encoded_dataset[["TOTAL_INCOME", "DAYS_EMPLOYED", "MONTHS_BALANCE"]])

#encoded_dataset.head()


## Training and Testing the Neural Network


In [5]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

X = encoded_dataset.drop(["STATUS"], axis=1)
Y = encoded_dataset[["STATUS"]]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=5)

mlp = MLPClassifier(solver = "sgd", random_state = 42, activation = "logistic", learning_rate_init = 0.3, hidden_layer_sizes = (6, 3), max_iter = 500)
mlp.fit(X_train, Y_train)
predictions = mlp.predict(X_test)

/home/null/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix

#Confusion Matrix
print("Confusion Matrix")
confusion_matrix(Y_test, predictions)

#Accuracy and MSE of model
print("\n Accuracy of Model: ", accuracy_score(Y_test, predictions))
print("Mean Squared Error: ", mean_squared_error(Y_test, predictions))

#Precision and Recall
print("\n Precision and Recall")
print(classification_report(Y_test, predictions))

Confusion Matrix

 Accuracy of Model:  0.989687328579265
Mean Squared Error:  0.010312671420735052

 Precision and Recall
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        94
         1.0       0.99      1.00      0.99      9021

    accuracy                           0.99      9115
   macro avg       0.49      0.50      0.50      9115
weighted avg       0.98      0.99      0.98      9115



/home/null/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/null/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/null/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [7]:
import pickle
#Saving the model for future use
FFNN = "FFNN_model.sav"
pickle.dump(mlp, open(FFNN, "wb"))

#Load the model for use
#premade_model = pickle.load(open("FFNN_model.sav", "rb"))
#result = premade_model.score(X_test, Y_test)
#print(result)